In [1]:
!pip install beautifulsoup4

In [2]:
import requests
base="https://pureportal.coventry.ac.uk"
robot=requests.get("https://pureportal.coventry.ac.uk/robots.txt")
robot_text=robot.text
print(robot_text)

User-Agent: *
Crawl-Delay: 1
Disallow: /*?*format=rss
Disallow: /*?*export=xls
Sitemap: https://pureportal.coventry.ac.uk/sitemap.xml


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
base="https://pureportal.coventry.ac.uk"
url = "https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting"
next_url=url+'/publications'
req=requests.get(next_url)
sup=BeautifulSoup(req.text,'html.parser')
url_list=[next_url]
page=0
while page<len(url_list):
    req=requests.get(url_list[page])
    sup=BeautifulSoup(req.text,'html.parser')
    page +=1
    for link in sup.select('li.next a'):
        url_list.append(base+link.get('href')) 
        print(base+link.get('href'))

https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=1
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=2
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=3
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=4
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=5
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=6
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=7
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=8
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-a

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
pub_data={'title':[],'author_name':[],'pub_date':[],'aut_link':[],'pub_link':[]}
def get_pub_links(url):
    for i in url_list:
        req=requests.get(i)
        sup=BeautifulSoup(req.text,'html.parser')
        links=sup.select('h3.title a')
        for link in links:
            pub_url=link.get('href')
            new_req=requests.get(pub_url)
            new_sup=BeautifulSoup(new_req.text,'html.parser')
            if new_sup.find('a',class_='link person') is not None:
                if new_sup.find('span',class_="type_classification") is not None:
                    if (new_sup.find('span',class_="type_classification").string)=='Paper' or \
                    (new_sup.find('span',class_="type_classification").string)=='Book':
                        for aut in new_sup.find_all('a',class_='link person'):
                            pub_data['author_name'].append(aut.string)
                            pub_data['aut_link'].append(aut.get('href'))
                            for title in new_sup.find('h1'):
                                pub_data['title'].append(title.string)
                                pub_data['pub_link'].append(pub_url)
                            for date in new_sup.select('tr.status td span.date'):
                                pub_data['pub_date'].append(date.string) 
    print(pub_data)

get_pub_links(url)

{'title': ['CEO Financial Experience and Firms’ Earnings Management in M&A: the importance of Experience Specificity', 'CEO Financial Experience and Firms’ Earnings Management in M&A: the importance of Experience Specificity', 'CEO Financial Experience and Firms’ Earnings Management in M&A: the importance of Experience Specificity', 'Finding our place in the world with the Sustainable Development Goals', 'Finding our place in the world with the Sustainable Development Goals', 'French Indochina War: Reflections for strategic resilience', 'French Indochina War: Reflections for strategic resilience', 'French Indochina War: Reflections for strategic resilience', 'Trade Wins or Trade Wars: The Perceptions and Knowledge in the Free Trade Debate', 'Earnings management by share-financed acquirers prior to deal announcements: The roles of financial expertise, tenure and reputation.', "Effects of director networks on acquiring firms' earnings management prior to M&A", "Effects of director networ

In [8]:
pubdata=pd.DataFrame(pub_data)
pubdata.head()

,title,author_name,pub_date,aut_link,pub_link
0,CEO Financial Experience and Firms’ Earnings M...,Thai Nguyen,7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/t...,https://pureportal.coventry.ac.uk/en/publicati...
1,CEO Financial Experience and Firms’ Earnings M...,Thang Nguyen,7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/t...,https://pureportal.coventry.ac.uk/en/publicati...
2,CEO Financial Experience and Firms’ Earnings M...,Panagiotis Andrikopoulos,7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/p...,https://pureportal.coventry.ac.uk/en/publicati...
3,Finding our place in the world with the Sustai...,Jaliyyah Bello,18 Feb 2021,https://pureportal.coventry.ac.uk/en/persons/j...,https://pureportal.coventry.ac.uk/en/publicati...
4,Finding our place in the world with the Sustai...,Luda Ruddock,18 Feb 2021,https://pureportal.coventry.ac.uk/en/persons/l...,https://pureportal.coventry.ac.uk/en/publicati...


## Removing punctuations in data

In [9]:
pubdata=pd.DataFrame(pub_data)
pubdata
title=pubdata['title']
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~’'''
for ele in title:
    for i in ele:
        if i in punc:
            title=list(map(lambda x: x.replace(i ,' '), title))
print(title)

['CEO Financial Experience and Firms  Earnings Management in M A  the importance of Experience Specificity', 'CEO Financial Experience and Firms  Earnings Management in M A  the importance of Experience Specificity', 'CEO Financial Experience and Firms  Earnings Management in M A  the importance of Experience Specificity', 'Finding our place in the world with the Sustainable Development Goals', 'Finding our place in the world with the Sustainable Development Goals', 'French Indochina War  Reflections for strategic resilience', 'French Indochina War  Reflections for strategic resilience', 'French Indochina War  Reflections for strategic resilience', 'Trade Wins or Trade Wars  The Perceptions and Knowledge in the Free Trade Debate', 'Earnings management by share financed acquirers prior to deal announcements  The roles of financial expertise  tenure and reputation ', 'Effects of director networks on acquiring firms  earnings management prior to M A', 'Effects of director networks on acqu

## Stop words

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download("stopwords")
nltk.download("punkt")
stpwd = stopwords.words('english')
print(stpwd)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pagid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pagid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Removing Stop words

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download("stopwords")
nltk.download("punkt")
sw = stopwords.words('english')
ps = PorterStemmer()
filtered_pub = []

for text in title:
        tokens = word_tokenize(text)
        tmp = ""
        for w in tokens:
            if w not in sw:
                tmp += ps.stem(w) + " "
        filtered_pub.append(tmp)
print(filtered_pub)

['ceo financi experi firm earn manag m a import experi specif ', 'ceo financi experi firm earn manag m a import experi specif ', 'ceo financi experi firm earn manag m a import experi specif ', 'find place world sustain develop goal ', 'find place world sustain develop goal ', 'french indochina war reflect strateg resili ', 'french indochina war reflect strateg resili ', 'french indochina war reflect strateg resili ', 'trade win trade war the percept knowledg free trade debat ', 'earn manag share financ acquir prior deal announc the role financi expertis tenur reput ', 'effect director network acquir firm earn manag prior m a ', 'effect director network acquir firm earn manag prior m a ', 'an empir review long term impact r d expenditur capit accumul gdp per capita ', 'compet institut logic institut embedded actor islam financi report standardis a compar studi ', 'complianc mercuri report provis minamata convent an assess develop countri perspect ', 'corpor environment account nigeria r

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pagid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pagid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Concatinating clean title in final DataFrame

In [12]:
new_filter=pd.DataFrame({"clean_title":filtered_pub})
pubdata_filter=pd.concat([pubdata,new_filter],axis=1,join='inner')
pubdata_filter.head()

,title,author_name,pub_date,aut_link,pub_link,clean_title
0,CEO Financial Experience and Firms’ Earnings M...,Thai Nguyen,7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/t...,https://pureportal.coventry.ac.uk/en/publicati...,ceo financi experi firm earn manag m a import ...
1,CEO Financial Experience and Firms’ Earnings M...,Thang Nguyen,7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/t...,https://pureportal.coventry.ac.uk/en/publicati...,ceo financi experi firm earn manag m a import ...
2,CEO Financial Experience and Firms’ Earnings M...,Panagiotis Andrikopoulos,7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/p...,https://pureportal.coventry.ac.uk/en/publicati...,ceo financi experi firm earn manag m a import ...
3,Finding our place in the world with the Sustai...,Jaliyyah Bello,18 Feb 2021,https://pureportal.coventry.ac.uk/en/persons/j...,https://pureportal.coventry.ac.uk/en/publicati...,find place world sustain develop goal
4,Finding our place in the world with the Sustai...,Luda Ruddock,18 Feb 2021,https://pureportal.coventry.ac.uk/en/persons/l...,https://pureportal.coventry.ac.uk/en/publicati...,find place world sustain develop goal


## Invert Indexing the text

In [13]:
list=[]
for text in pubdata_filter["clean_title"].str.split():
    list.append(text)
for i in list:
        indexing={word: [(list.index(i))+1] for word in i}
print(indexing)

{'human': [49], 'capit': [49], 'trade': [49], 'fdi': [49], 'econom': [49], 'growth': [49], 'thailand': [49], 'caus': [49]}


In [14]:
from collections import defaultdict
docs=[sent for sent in pubdata_filter.clean_title.str.split()]
titleIndex = defaultdict(set)
for index, text in enumerate(docs):
    for word in text:
        titleIndex[word].add(index)
print(titleIndex)

docs_aut=[sent for sent in pubdata_filter.author_name.str.split()]
authorIndex = defaultdict(set)
for index, text in enumerate(docs_aut):
    for word in text:
        authorIndex[word].add(index)
print(authorIndex)

docs_title=[sent for sent in pubdata_filter.title.str.split()]
TitleIndex = defaultdict(set)
for index, text in enumerate(docs_title):
    for word in text:
        TitleIndex[word].add(index)
print(TitleIndex)

defaultdict(<class 'set'>, {'ceo': {0, 1, 2}, 'financi': {0, 1, 2, 37, 9, 13, 17, 23, 27, 29, 31}, 'experi': {0, 1, 2}, 'firm': {0, 1, 2, 10, 11}, 'earn': {0, 1, 2, 9, 10, 11, 21}, 'manag': {0, 1, 2, 9, 10, 11, 17, 21}, 'm': {0, 1, 2, 10, 11}, 'a': {0, 1, 2, 34, 35, 37, 38, 10, 11, 42, 13, 45, 16, 20, 22, 25, 26}, 'import': {0, 1, 2}, 'specif': {0, 1, 2}, 'find': {3, 4}, 'place': {3, 4}, 'world': {3, 4}, 'sustain': {18, 3, 4}, 'develop': {3, 4, 20, 14}, 'goal': {3, 4}, 'french': {5, 6, 7}, 'indochina': {5, 6, 7}, 'war': {35, 5, 6, 7, 8}, 'reflect': {5, 6, 7}, 'strateg': {5, 6, 7}, 'resili': {5, 6, 7}, 'trade': {8, 48}, 'win': {8}, 'the': {38, 8, 9, 25, 26}, 'percept': {8}, 'knowledg': {8}, 'free': {8}, 'debat': {8}, 'share': {9}, 'financ': {34, 36, 39, 40, 9, 16, 22, 25, 26}, 'acquir': {9, 10, 11}, 'prior': {9, 10, 11, 21}, 'deal': {9}, 'announc': {9, 21}, 'role': {9, 15}, 'expertis': {9}, 'tenur': {9}, 'reput': {9}, 'effect': {10, 11}, 'director': {10, 11}, 'network': {10, 11}, 'an': 

## Query

In [1]:
query=input("Search ")
key=titleIndex.keys()
if query in key:
    for i in titleIndex[query]:
#         print('\n',pubdata_filter.loc[i,:])
        print('\n The Title:',pubdata_filter.loc[i,"title"])
        print('\n Author Name:',pubdata_filter.loc[i,"author_name"])
        print("\n Author link:",pubdata_filter.loc[i,'aut_link'])
        print("\n publication link:",pubdata_filter.loc[i,'pub_link'])
        print("\n publication date:", pubdata_filter.loc[i,'pub_date'])
key=authorIndex.keys()
if query in key:
    for i in authorIndex[query]:
#         print('\n',pubdata_filter.loc[i,:])
        print('\n The Title:',pubdata_filter.loc[i,"title"])
        print('\n Author Name:',pubdata_filter.loc[i,"author_name"])
        print("\n Author link:",pubdata_filter.loc[i,'aut_link'])
        print("\n publication link:",pubdata_filter.loc[i,'pub_link'])
        print("\n publication date:", pubdata_filter.loc[i,'pub_date'])
        
key=TitleIndex.keys()
if query in key:
    for i in TitleIndex[query]:
#         print('\n',pubdata_filter.loc[i,:])
        print('\n The Title:',pubdata_filter.loc[i,"title"])
        print('\n Author Name:',pubdata_filter.loc[i,"author_name"])
        print("\n Author link:",pubdata_filter.loc[i,'aut_link'])
        print("\n publication link:",pubdata_filter.loc[i,'pub_link'])
        print("\n publication date:", pubdata_filter.loc[i,'pub_date'])

Search ceo


NameError: name 'titleIndex' is not defined

## Task-2

In [16]:
import pandas as pd
data=pd.read_csv('data.csv')
data

,category,text
0,sport,Lucy Bronze has revealed she plays through the...
1,sport,"The 30-year-old right-back, who joined Barcelo..."
2,sport,She scored in England's 4-0 semi-final win ove...
3,sport,It's been difficult to come back from a knee i...
4,sport,There are plenty of players who are having to ...
...,...,...
104,health,new subvariants do appear to be spreading more...
105,health,Doctors say it is vital people get the recomme...
106,health,maximum protection against existing and emergi...
107,health,How quickly could we get new vaccines against ...


In [121]:
data['category'].unique()

array(['sport', 'politics', 'health'], dtype=object)

## Removing Punctuation

In [17]:
data_text=data['text']
punc = '''!()-[]{};:'"\,''<>./?@#$%^&*_~’``'''
no_punc_text=data_text.apply(lambda x: ''.join(i for i in x if i not in set(punc)))
print(no_punc_text)


0      Lucy Bronze has revealed she plays through the...
1      The 30yearold rightback who joined Barcelona f...
2      She scored in Englands 40 semifinal win over S...
3      Its been difficult to come back from a knee in...
4      There are plenty of players who are having to ...
                             ...                        
104    new subvariants do appear to be spreading more...
105    Doctors say it is vital people get the recomme...
106    maximum protection against existing and emergi...
107    How quickly could we get new vaccines against ...
108    The bug called Burkholderia pseudomallei can m...
Name: text, Length: 109, dtype: object


In [18]:
no_num=no_punc_text.str.lower().replace("[0-9]"," ")
data['no_punc_num']=no_num.apply(lambda x:' '.join(x.split()))
data

,category,text,no_punc_num
0,sport,Lucy Bronze has revealed she plays through the...,lucy bronze has revealed she plays through the...
1,sport,"The 30-year-old right-back, who joined Barcelo...",the 30yearold rightback who joined barcelona f...
2,sport,She scored in England's 4-0 semi-final win ove...,she scored in englands 40 semifinal win over s...
3,sport,It's been difficult to come back from a knee i...,its been difficult to come back from a knee in...
4,sport,There are plenty of players who are having to ...,there are plenty of players who are having to ...
...,...,...,...
104,health,new subvariants do appear to be spreading more...,new subvariants do appear to be spreading more...
105,health,Doctors say it is vital people get the recomme...,doctors say it is vital people get the recomme...
106,health,maximum protection against existing and emergi...,maximum protection against existing and emergi...
107,health,How quickly could we get new vaccines against ...,how quickly could we get new vaccines against ...


In [21]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stpwrds = stopwords.words('english')
print(stpwrds)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pagid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

ps = PorterStemmer()
clean_docs = []
for text in data['no_punc_num'].str.split():
    tmp = ""
    for word in text:
        if word not in stpwrds:
            tmp += ps.stem(word) + " "
    clean_docs.append(tmp)

print(clean_docs)

['luci bronz reveal play pain knee injuri ', '30yearold rightback join barcelona manchest citi summer ', 'score england 40 semifin win sweden bramal lane ', 'difficult come back knee injuri linger long time ', 'plenti player play pain career ', 'also three champion leagu row lyon ', 'everyon keep say doesnt look like she struggl ', 'luci bronz coupl year ago best player world ', 'bronz set beth mead open sweden head second goal ', 'still happi contribut team still play good footbal ', 'goal assist someth consid major part game ', 'ellen white lauren hemp get scoresheet ', 'score euro ive score world cup ', 'goal came realli import time straight halftim kind kill ', 'england face winner thursday semifin franc germani wembley ', 'lot work get moment firmli believ manag ', 'manchest unit look forward see develop help team ', 'coverag manchest unit bigger better ever here everyth ', 'media interest grew journalist realis brother call ping ', 'ho appear cope admir attent expect ', 'never re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pagid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
data['clean_text']=clean_docs
final_data=data[['clean_text','category']]
final_data

,clean_text,category
0,luci bronz reveal play pain knee injuri,sport
1,30yearold rightback join barcelona manchest ci...,sport
2,score england 40 semifin win sweden bramal lane,sport
3,difficult come back knee injuri linger long time,sport
4,plenti player play pain career,sport
...,...,...
104,new subvari appear spread easili,health
105,doctor say vital peopl get recommend number do...,health
106,maximum protect exist emerg variant,health
107,quickli could get new vaccin variant,health


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer =TfidfVectorizer()
vect = vectorizer.fit_transform(final_data['clean_text'],final_data['category'])
print(vect.todense())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [25]:
from sklearn.cluster import KMeans
K = 3
model = KMeans(n_clusters=K, init='k-means++', max_iter=100, n_init=1)
model.fit(vect)

print("cluster no. of input documents, in the order they received:")
print(model.labels_)


cluster no. of input documents, in the order they received:
[1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 2 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 0 2 2 0 2 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 2 2 0 1 0 0 0 1 0 0 0 0 2 0 1 2 1 2 0 2 2 0 2 0]


In [297]:
Y = vectorizer.transform(["Bronze has revealed she plays through"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["Rishi Sunak: 'There are millions of homes that need better insulation"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["to give working women more rights"])
prediction = model.predict(Y)
print(prediction)


[0]
[1]
[2]


In [ ]:
for i in range (0,13):\n",
    "    tag = \"?page=\"+str(i)\n",
    "    url = f'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/{tag}'\n",
    "    response = requests.get(url)\n",
    "    soup = BeautifulSoup(response.content, 'html.parser')\n",
    "\n",
    "    for publications in soup.findAll(\"div\",{\"class\":\"result-container\"}):\n",
    "        for data in publications.findAll(\"a\",{\"class\":\"link person\"}):\n",
    "            name1 = data.string\n",
    "            link1 = data.get('href')\n",
    "            a_name.append(name1)\n",
    "            a_link.append(link1)\n",
    "\n",
    "        for item in publications.find_all(\"h3\",class_=\"title\"):\n",
    "            for var in item.find_all(\"a\",class_=\"link\"):\n",
    "                name = var.string\n",
    "                link = var.get('href')\n",
    "                publication_title_list.append(name)\n",
    "                publication_links.append(link)\n",
    "        for values in publications.find_all(\"span\",class_=\"date\"):\n",
    "            d = values.string\n",
      date_list.append(d)\n",
    "print(\"Done crawling all data\")\n",
    "           "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 119,
   "id": "3a5944ad",
   "metadata": {},
   "outputs": [],
   "source": [
    "#Converting into DF\n",
    "from pandas import DataFrame\n"
   ]
  },
  {
   cell_type": "code",
   execution_count": 120,
   id: "af03ab87",
   metadata": {},
   outputs": [],
   source": [
    bookname = DataFrame(publication_title_list,columns = ['Bookname'] , index = (i for i in range(1,(len(publication_title_list)+1))))\n",,
    bookurl = DataFrame(publication_links,columns = ['Publication Links'] , index = (i for i in range(1,(len(publication_links)+1))))\n",
    authorname = DataFrame(a_name,columns = ['Author Name'] , index = (i for i in range(1,(len(a_name)+1))))\n",
    "\n",
    "linksof = DataFrame(a_link,columns = ['Author Links'] , index = (i for i in range(1,(len(a_name)+1))))\n",
    "\n",
    "dates = DataFrame(date_list,columns = ['Date'] , index = (i for i in range(1,(len(date_list)+1))))\n"